In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from PyPDF2 import PdfReader
import gradio as gr

from loguru import logger

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [ ]:
def push(message, device=None, timeout=5):
    if not all([pushover_user, pushover_token, pushover_url]):
        logger.error("Pushover configuration is missing.")
        return False

    logger.info("Sending push notification...")
    
    payload = {
        "user": pushover_user,
        "token": pushover_token,
        "message": message,
    }
    if device:
        payload["device"] = device

    try:
        response = requests.post(pushover_url, data=payload, timeout=timeout)
        if response.status_code != 200:
            logger.error(f"Pushover failed: {response.status_code} {response.text}")
            return False
        logger.info("Push notification sent successfully.")
        return True
    except requests.RequestException as e:
        logger.exception(f"Error sending push notification: {e}")
        return False

In [6]:
push("hello world")

2025-08-11 14:53:48.708 | INFO     | __main__:push:6 - Sending push notification...
2025-08-11 14:53:49.024 | INFO     | __main__:push:22 - Push notification sent successfully.


True

In [7]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [8]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorder": "ok"}

In [9]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [10]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that could not be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [11]:
tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json}]

logger.info(f"Tools: {tools}")

2025-08-11 21:24:40.832 | INFO     | __main__:<module>:5 - Tools: [{'type': 'function', 'function': {'name': 'record_user_details', 'description': 'Use this tool to record that a user is interested in being in touch and provided an email address', 'parameters': {'type': 'object', 'properties': {'email': {'type': 'string', 'description': 'The email address of this user'}, 'name': {'type': 'string', 'description': "The user's name, if they provided it"}, 'notes': {'type': 'string', 'description': "Any additional information about the conversation that's worth recording"}}, 'required': ['email'], 'additionalProperties': False}}}, {'type': 'function', 'function': {'name': 'record_unknown_question', 'description': "Always use this tool to record any question that couldn't be answered", 'parameters': {'type': 'object', 'properties': {'question': {'type': 'string', 'description': 'The question that could not be answered'}}, 'required': ['question'], 'additionalProperties': False}}}]


In [13]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        logger.info(f'Tool called: {tool_name}')
        
        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
            
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
        
    return results

In [14]:
# more elegant way
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        logger.info(f'Tool called: {tool_name}')
        
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
            
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
        
    return results

In [16]:
reader = PdfReader("../../1_foundations/me/linkedin.pdf")
linkedid = ""

pages = [page.extract_text() for page in reader.pages]
linkedin = " ".join(pages)

with open("../../1_foundations/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    
name = "Ed Donner"

In [17]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [32]:
def chat(message, history):
    messages = [{
        "role": "system",
        "content": system_prompt
    }] + history + [{
        "role": "user",
        "content": message
    }]
    
    done = False
    while not done:
        response = openai.chat.completions.create(
            model='gpt-4o-mini',
            messages=messages,
            tools=tools
        )
        
        if response.choices[0].finish_reason == 'tool_calls':
            message = response.choices[0].message
            results = handle_tool_calls(message.tool_calls)
            messages.append(message)
            messages.extend(results)
        
        else:
            done = True
            
    logger.info(f"Messages: {messages}")
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


2025-08-11 22:02:07.856 | INFO     | __main__:handle_tool_calls:7 - Tool called: record_unknown_question
2025-08-11 22:02:07.860 | INFO     | __main__:push:6 - Sending push notification...
2025-08-11 22:02:08.572 | INFO     | __main__:push:22 - Push notification sent successfully.
2025-08-11 22:02:10.521 | INFO     | __main__:chat:27 - Messages: [{'role': 'system', 'content': "You are acting as Ed Donner. You are answering questions on Ed Donner's website, particularly questions related to Ed Donner's career, background, skills and experience. Your responsibility is to represent Ed Donner for interactions on the website as faithfully as possible. You are given a summary of Ed Donner's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer to any question, use your record_unknown_question tool to record the question that you could